In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt 
df = pd.read_excel("World Health Data.xlsx")
df = df[['Country','MortalityRate','Population','Year','kids per woman','Income Group']]
df['kids_per_woman'] = df['kids per woman']
df.drop(columns=['kids per woman'], inplace=True)
df.head()

,Country,MortalityRate,Population,Year,Income Group,kids_per_woman
0,Afghanistan,356.5,9164945.0,1961.0,Low,7.45
1,Afghanistan,350.6,9343772.0,1962.0,Low,7.45
2,Afghanistan,345.0,9531555.0,1963.0,Low,7.45
3,Afghanistan,339.7,9728645.0,1964.0,Low,7.45
4,Afghanistan,334.1,9935358.0,1965.0,Low,7.45


In [2]:
import linearmodels as plm
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Income_Group'] = label_encoder.fit_transform(df['Income Group'])
df = df.set_index(['Country', 'Year'], drop=False)

reg_re = plm.RandomEffects.from_formula(
    formula='MortalityRate ~ Population + kids_per_woman + C(Income_Group)', data=df)
results_re = reg_re.fit()

table_re = pd.DataFrame({'b': round(results_re.params, 4),
                         'se': round(results_re.std_errors, 4),
                         't': round(results_re.tstats, 4),
                         'pval': round(results_re.pvalues, 4)})
print(f'table_re: \n{table_re}\n')
original_income_groups = label_encoder.classes_
for i in range(len(original_income_groups)):
    income_group = original_income_groups[i]
    coefficient = f"C(Income_Group)[T.{i}]"
    print(f"{income_group} corresponds to {coefficient}")

table_re: 
                            b      se         t   pval
Population            -0.0000  0.0000  -13.5522  0.000
kids_per_woman        30.2896  0.2725  111.1679  0.000
C(Income_Group)[T.0] -50.7073  4.1749  -12.1456  0.000
C(Income_Group)[T.1] -28.3475  3.6970   -7.6678  0.000
C(Income_Group)[T.2] -50.8065  4.2332  -12.0020  0.000
C(Income_Group)[T.3]  -5.8892  4.7496   -1.2399  0.215

High corresponds to C(Income_Group)[T.0]
Low corresponds to C(Income_Group)[T.1]
Medium corresponds to C(Income_Group)[T.2]
nan corresponds to C(Income_Group)[T.3]


/Users/zhangyuanzhuo/anaconda3/lib/python3.11/site-packages/linearmodels/panel/model.py:2616: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


The regression results indicate several significant associations between predictor variables and mortality rates. Firstly, the coefficient for population size is negative and statistically significant, suggesting that as population increases, mortality rates tend to decrease. Similarly, the coefficient for fertility rates, as measured by 'kids_per_woman', is positive and highly significant, indicating that higher fertility rates are associated with higher mortality rates. Moreover, the coefficients for income groups demonstrate substantial variation and significance. Specifically, when compared to the reference category, 'High' income group exhibits a significantly lower mortality rate, whereas 'Low' and 'Medium' income groups also show lower mortality rates, albeit to a lesser extent. Interestingly, the 'nan' category, representing missing values, does not exhibit a significant difference in mortality rate compared to the reference category, suggesting that the effect of missing income data on mortality rate is not statistically significant. Overall, these findings highlight the complex interplay between demographic factors, such as population size and fertility rates, and socioeconomic determinants, such as income levels, in influencing mortality rates across different income groups. 